In [12]:
import numpy as np
import sys, os
sys.path.append(os.pardir + '/deep-learning-from-scratch/') 
from dataset.mnist import load_mnist
from PIL import Image
import pickle 
from common.layers import *
from common.functions import *
from common.gradient import numerical_gradient
from tqdm import tqdm
from collections import OrderedDict

In [27]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std= 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['b2'] = np.zeros(output_size)

        # レイヤの作成
        self.Layers = OrderedDict()
        self.Layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.Layers['Relu1'] = Relu()
        self.Layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.Layers.values():
            x = layer.forward(x)
        return x


    # # 推論を行う
    # # xは画像データ
    # def predict(self, x):
    #     # 変数の定義
    #     W1, W2 = self.params['W1'], self.params['W2']
    #     b1, b2 = self.params['b1'], self.params['b2']
        
    #     # 各層の演算
    #     a1 = np.dot(x, W1) + b1
    #     z1 = sigmoid(a1)
    #     a2 = np.dot(z1, W2) + b2
    #     y = softmax(a2)

    #     return y

    # 損失関数を求める
    # x:画像データ、t:正解ラベル
    def loss(self, x, t):
        y = self.predict(x)
        loss = self.lastLayer.forward(y, t)
        return loss
    
    # 精度算出
    def accurancy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accurancy = np.sum(y == t)/float(x.shape[0])
        return accurancy
    
    # 重みパラメータの勾配を求める
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads


    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        Layers = list(self.Layers.values())
        Layers.reverse()
        # 勾配の計算
        for layer in Layers:
            dout = layer.backward(dout)

        # 重みとバイアスの勾配の収集
        grads = {}
        grads['W1'] = self.Layers['Affine1'].dW
        grads['b1'] = self.Layers['Affine1'].db
        grads['W2'] = self.Layers['Affine2'].dW
        grads['b2'] = self.Layers['Affine2'].db

        return grads

In [ ]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
    
    def update(self, params, grads):
        for key in params.key():
            params[key] -= self.lr * grads[key]

class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)

        for key in params.key():
            # SGDは勾配が大きい方向に急激に移動する傾向があり、最適点を行き過ぎることがある
            # モーメンタムは、このような振動を抑制し、より滑らかなパラメータの更新を可能にする
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

# AdaGradは勾配の大きさ毎に学習率を可変にする手法
# 学習が進めば進むほど更新度合いは小さくなる
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
    
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)

        for key in params.key():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr*grads[key]/(np.sort(self.h[key]) + 1e-7)

# AdamはAdaGradとMomentumを組み合わせた手法


In [28]:
(x_train, y_train), (x_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

#　ハイパーパラメータ
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
lr = 0.01

train_loss_list = []
train_acc_list = []
test_acc_list = []

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
optimizer = SGD()

iter_per_epoch = max(train_size / batch_size, 1)

for i in tqdm(range(iters_num)):
    # ミニバッチの取得
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    y_batch = y_train[batch_mask]

    # 勾配の計算
    grad = network.gradient(x_batch, y_batch)
    params = network.params
    optimizer.update(params, grad)

    # # パラメータの更新
    # for key in ('W1', 'W2', 'b1', 'b2'):
    #     network.params[key] -= lr*grad[key]

    # 学習経過の記録
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accurancy(x_train, y_train)
        test_acc = network.accurancy(x_test, y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)



  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 33/10000 [00:00<01:59, 83.56it/s] 

0.1066 0.1081


  7%|▋         | 658/10000 [00:02<00:50, 184.54it/s]

0.5368 0.5437


...

 91%|█████████ | 9066/10000 [00:32<00:04, 190.65it/s]

0.9149666666666667 0.9179


 97%|█████████▋| 9654/10000 [00:34<00:01, 187.47it/s]

0.9162 0.9194


100%|██████████| 10000/10000 [00:35<00:00, 279.67it/s]


In [7]:
y_batch

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 